# 1. Data pre-prossesing

#### 1.1 clean up unnesessary files and convert to kitti dataset format

In [ ]:
# sit to kitti convert

import os
import shutil
from tqdm import tqdm


def cleanup_dataset(root_dir):
    folders_to_delete = ['bev', 'ego_trajectory', 'imu', 'label_2d', 'label_3d', 'rtk', 'tf']

    print("Cleaning up dataset...")
    for folder in folders_to_delete:
        for dirpath, dirnames, filenames in os.walk(root_dir):
            if folder in dirnames:
                folder_path = os.path.join(dirpath, folder)
                shutil.rmtree(folder_path, ignore_errors=True)
                # print(f"Deleted folder: {folder_path}")

    for dirpath, dirnames, filenames in os.walk(root_dir):
        if 'cam_img' in dirnames:
            cam_img_path = os.path.join(dirpath, 'cam_img')
            for subdir in os.listdir(cam_img_path):
                if subdir != '4':
                    subdir_path = os.path.join(cam_img_path, subdir)
                    if os.path.isdir(subdir_path):
                        shutil.rmtree(subdir_path, ignore_errors=True)
                        # print(f"Deleted folder: {subdir_path}")

            if '4' in os.listdir(cam_img_path):
                data_rgb_path = os.path.join(cam_img_path, '4', 'data_rgb')
                if os.path.isdir(data_rgb_path):
                    shutil.rmtree(data_rgb_path, ignore_errors=True)
                    # print(f"Deleted folder: {data_rgb_path}")

                data_undist_path = os.path.join(cam_img_path, '4', 'data_undist')
                if os.path.isdir(data_undist_path):
                    new_path = os.path.join(dirpath, 'image_02')
                    os.rename(data_undist_path, new_path)
                    # print(f"Renamed folder: {data_undist_path} to {new_path}")

        if 'velo' in dirnames:
            velo_path = os.path.join(dirpath, 'velo')
            for subdir in os.listdir(velo_path):
                if subdir != 'concat':
                    subdir_path = os.path.join(velo_path, subdir)
                    if os.path.isdir(subdir_path):
                        shutil.rmtree(subdir_path, ignore_errors=True)
                        # print(f"Deleted folder: {subdir_path}")

    # After processing, remove cam_img folder if it exists
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if 'cam_img' in dirnames:
            cam_img_path = os.path.join(dirpath, 'cam_img')
            shutil.rmtree(cam_img_path, ignore_errors=True)
            # print(f"Deleted folder: {cam_img_path}")

    # Remove data folder inside velo/concat folder
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if 'velo' in dirnames:
            velo_path = os.path.join(dirpath, 'velo')
            concat_path = os.path.join(velo_path, 'concat')
            data_path = os.path.join(concat_path, 'data')
            if os.path.isdir(data_path):
                shutil.rmtree(data_path, ignore_errors=True)
                # print(f"Deleted folder: {data_path}")

    # Rename bin_data folder to data inside velo/concat
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if 'velo' in dirnames:
            velo_path = os.path.join(dirpath, 'velo')
            concat_path = os.path.join(velo_path, 'concat')
            bin_data_path = os.path.join(concat_path, 'bin_data')
            if os.path.isdir(bin_data_path):
                new_data_path = os.path.join(concat_path, 'data')
                os.rename(bin_data_path, new_data_path)
                # print(f"Renamed folder: {bin_data_path} to {new_data_path}")

def move_image_files(root_dir):
    print("Moving image files...")
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if 'image_02' in dirnames:
            image_02_path = os.path.join(dirpath, 'image_02')
            data_path = os.path.join(image_02_path, 'data')
            os.makedirs(data_path, exist_ok=True)

            files_to_move = [f for f in os.listdir(image_02_path) if os.path.isfile(os.path.join(image_02_path, f))]
            for file in tqdm(files_to_move, desc=f"Moving files in {image_02_path}", unit="files"):
                shutil.move(os.path.join(image_02_path, file), os.path.join(data_path, file))


def rename_velo_folder(root_dir):
    print("Renaming velo folders...")
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if 'velo' in dirnames:
            velo_path = os.path.join(dirpath, 'velo')
            new_velo_path = os.path.join(dirpath, 'velodyne_points')
            os.rename(velo_path, new_velo_path)
            # print(f"Renamed folder: {velo_path} to {new_velo_path}")


def rename_files(root_dir):
    print("Renaming files...")
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.bin') or filename.endswith('.png'):
                base, ext = os.path.splitext(filename)
                try:
                    new_base = f"{int(base):010d}"
                except ValueError:
                    continue
                new_name = f"{new_base}{ext}"
                old_file = os.path.join(dirpath, filename)
                new_file = os.path.join(dirpath, new_name)
                
                # Check if new_file already exists
                if os.path.exists(new_file):
                    print(f"File {new_file} already exists. Skipping rename for {old_file}")
                    continue
                
                os.rename(old_file, new_file)
                # print(f"Renamed file: {old_file} to {new_file}")

def move_concat_data_folder(root_dir):
    print("Moving concat data folder and deleting concat folder...")
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if 'velodyne_points' in dirnames:
            velodyne_points_path = os.path.join(dirpath, 'velodyne_points')
            concat_path = os.path.join(velodyne_points_path, 'concat')
            data_path = os.path.join(concat_path, 'data')
            if os.path.isdir(data_path):
                # Move data folder to velodyne_points_path
                new_data_path = os.path.join(velodyne_points_path, 'data')
                shutil.move(data_path, new_data_path)
                # Delete concat folder
                shutil.rmtree(concat_path, ignore_errors=True)



if __name__ == "__main__":
    root_dir = "/media/eddie/Samsung_T5/test"
    cleanup_dataset(root_dir)
    move_image_files(root_dir)
    rename_velo_folder(root_dir)
    move_concat_data_folder(root_dir)
    rename_files(root_dir)


#### 1.2 convert sit dataset split data format to kitti dataset

In [ ]:
# train.txt -> train_files_sit.txt

def convert_to_train_files_format(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        lines = infile.readlines()
        for line in lines:
            components = line.strip().split('*')
            if len(components) == 3:
                scene, drive, frame = components
                new_line = f"{scene}/{drive} {frame} l\n"
                outfile.write(new_line)

input_files = [
    '/media/eddie/28901C58901C2F36/dataset/SiT_Dataset_full_kitti/train.txt',
    '/media/eddie/28901C58901C2F36/dataset/SiT_Dataset_full_kitti/val.txt',
    '/media/eddie/28901C58901C2F36/dataset/SiT_Dataset_full_kitti/test.txt'
]

output_files = [
    '/media/eddie/28901C58901C2F36/dataset/SiT_Dataset_full_kitti/train_files_sit.txt',
    '/media/eddie/28901C58901C2F36/dataset/SiT_Dataset_full_kitti/val_files_sit.txt',
    '/media/eddie/28901C58901C2F36/dataset/SiT_Dataset_full_kitti/test_files_sit.txt'
]

convert_to_train_files_format(input_files, output_files)

## 2. Detph Map GT generation

#### 2.1 Depth map GT generated directly projection from lidar point cloud

In [ ]:
# Front point cloud projected to cam4(front, P3)

import numpy as np
import cv2
import os
import concurrent.futures

def load_calibration(calib_file):
    calib_data = {}
    with open(calib_file) as f:
        for line in f:
            if line.startswith("P3_intrinsic:"):
                calib_data['P3_intrinsic'] = np.array([float(x) for x in line.split()[1:]]).reshape(3, 3)
            elif line.startswith("P3_extrinsic:"):
                calib_data['P3_extrinsic'] = np.array([float(x) for x in line.split()[1:]]).reshape(3, 4)
    return calib_data

def load_point_cloud(bin_file):
    points = np.fromfile(bin_file, dtype=np.float32).reshape(-1, 4)
    return points[:, :3]

def transform_point_cloud(points, R, T):
    points_hom = np.hstack((points, np.ones((points.shape[0], 1))))
    transformation_matrix = np.hstack((R, T))
    transformation_matrix = np.vstack((transformation_matrix, [0, 0, 0, 1]))
    points_transformed = points_hom.dot(transformation_matrix.T)
    return points_transformed[:, :3]

# filter out backward points
def filter_forward_points(points):
    return points[points[:, 2] > 0]

def project_to_image(points, P):
    points_hom = np.hstack((points, np.ones((points.shape[0], 1))))
    points_2d = points_hom.dot(P.T)
    points_2d[:, :2] /= points_2d[:, 2][:, np.newaxis]
    return points_2d[:, :2], points_2d[:, 2]

def create_depth_map(points_2d, depth, image_size):
    depth_map = np.zeros(image_size)
    for i, (x, y) in enumerate(points_2d):
        if 0 <= x < image_size[1] and 0 <= y < image_size[0]:
            if depth_map[int(y), int(x)] == 0 or depth_map[int(y), int(x)] > depth[i]:
                depth_map[int(y), int(x)] = depth[i]
    return depth_map

def refine_depth_map(depth_map, inpaint_radius=3):
    depth_map[depth_map == 0] = np.nan
    refined_map = cv2.inpaint(depth_map.astype(np.float32), np.isnan(depth_map).astype(np.uint8), inpaint_radius, cv2.INPAINT_NS)
    return refined_map

def process_image(calib, img_file, lidar_file, output_dir, inpaint_radius):
    P3_intrinsic = calib['P3_intrinsic']
    P3_extrinsic = calib['P3_extrinsic']
    
    R = P3_extrinsic[:, :3]
    T = P3_extrinsic[:, 3:4]
    P = np.hstack((P3_intrinsic, np.zeros((3, 1))))
    
    points = load_point_cloud(lidar_file)
    points = filter_forward_points(points)
    points_cam = transform_point_cloud(points, R, T)
    points_2d, depth = project_to_image(points_cam, P)
    
    image = cv2.imread(img_file)
    image_size = image.shape[:2]
    
    depth_map = create_depth_map(points_2d, depth, image_size)
    
    # Refinement with adjustable inpaint radius
    depth_map_refined = refine_depth_map(depth_map, inpaint_radius=inpaint_radius)
    
    output_path = os.path.join(output_dir, os.path.basename(img_file))
    cv2.imwrite(output_path, depth_map_refined)
    # print(f"Saved refined depth map for {img_file} at {output_path}")

def process_folder(base_dir, inpaint_radius=3):
    calib_file = os.path.join(base_dir, 'calib', '0.txt')
    image_dir = os.path.join(base_dir, 'image_02', 'data')
    lidar_dir = os.path.join(base_dir, 'velodyne_points', 'data')
    output_dir = os.path.join(base_dir, 'proj_depth', 'groundtruth', 'image_02')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    calib = load_calibration(calib_file)
    
    image_files = sorted(os.listdir(image_dir))
    lidar_files = sorted(os.listdir(lidar_dir))
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for img_file, lidar_file in zip(image_files, lidar_files):
            img_path = os.path.join(image_dir, img_file)
            lidar_path = os.path.join(lidar_dir, lidar_file)
            futures.append(executor.submit(process_image, calib, img_path, lidar_path, output_dir, inpaint_radius))
        for future in concurrent.futures.as_completed(futures):
            future.result()

def main(root_dir, inpaint_radius=3):
    for subdir, dirs, files in os.walk(root_dir):
        if 'calib' in dirs and 'image_02' in dirs and 'velodyne_points' in dirs:
            print(f"Processing folder: {subdir}")
            process_folder(subdir, inpaint_radius=inpaint_radius)

root_dir = '/home/eddie/depth_estimation/monodepth2/sit_test_r3'
inpaint_radius = 2  # Adjust this value to refine the depth map less or more
main(root_dir, inpaint_radius=inpaint_radius)


#### 2.2 Depth map GT using interpolation method

In [ ]:
import numpy as np
import cv2
import os
import concurrent.futures
from scipy.interpolate import griddata

def load_calibration(calib_file):
    calib_data = {}
    with open(calib_file) as f:
        for line in f:
            key, value = line.split(':', 1)
            calib_data[key] = np.array([float(x) for x in value.split()])
    return calib_data

def load_point_cloud(bin_file):
    points = np.fromfile(bin_file, dtype=np.float32).reshape(-1, 4)
    return points[:, :3]

def transform_and_filter_points(points, calib):
    P3 = calib['P3_intrinsic'].reshape(3, 3)
    R0_rect = calib['R0_rect'].reshape(3, 3)
    P3_extrinsic = calib['P3_extrinsic'].reshape(3, 4)
    
    points = np.hstack((points, np.ones((points.shape[0], 1))))
    points = np.dot(points, P3_extrinsic.T)
    points = np.dot(points, R0_rect.T)
    
    points = points[points[:, 2] > 0]
    
    points = np.dot(points, P3.T)
    
    points[:, 0] /= points[:, 2]
    points[:, 1] /= points[:, 2]
    
    return points[:, :2], points[:, 2]

def create_depth_map(points_2d, depth, image_size, point_size=1):
    depth_map = np.full(image_size, np.nan)
    for i, (x, y) in enumerate(points_2d):
        if 0 <= x < image_size[1] and 0 <= y < image_size[0]:
            for dx in range(-point_size, point_size + 1):
                for dy in range(-point_size, point_size + 1):
                    if 0 <= x + dx < image_size[1] and 0 <= y + dy < image_size[0]:
                        if np.isnan(depth_map[int(y + dy), int(x + dx)]) or depth_map[int(y + dy), int(x + dx)] > depth[i]:
                            depth_map[int(y + dy), int(x + dx)] = depth[i]
    return depth_map

def interpolate_depth_map(depth_map, method='linear'):
    x = np.arange(depth_map.shape[1])
    y = np.arange(depth_map.shape[0])
    xx, yy = np.meshgrid(x, y)
    known_points = np.array(np.where(~np.isnan(depth_map))).T
    known_values = depth_map[~np.isnan(depth_map)]
    interpolated_map = griddata(known_points, known_values, (yy, xx), method=method, fill_value=0)
    return interpolated_map

def process_image(calib, img_file, lidar_file, output_dir, interpolation_method, point_size):
    points = load_point_cloud(lidar_file)
    points_2d, depth = transform_and_filter_points(points, calib)
    
    image = cv2.imread(img_file)
    image_size = image.shape[:2]
    
    depth_map = create_depth_map(points_2d, depth, image_size, point_size=point_size)
    
    # Interpolation with adjustable method
    depth_map_interpolated = interpolate_depth_map(depth_map, method=interpolation_method)
    
    output_path = os.path.join(output_dir, os.path.basename(img_file))
    cv2.imwrite(output_path, (depth_map_interpolated * 256).astype(np.uint16))
    # print(f"Saved interpolated depth map for {img_file} at {output_path}")

def process_folder(base_dir, interpolation_method='linear', point_size=1):
    calib_file = os.path.join(base_dir, 'calib', '0.txt')
    image_dir = os.path.join(base_dir, 'image_02', 'data')
    lidar_dir = os.path.join(base_dir, 'velodyne_points', 'data')
    output_dir = os.path.join(base_dir, 'proj_depth', 'groundtruth', 'image_02')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    calib = load_calibration(calib_file)
    
    image_files = sorted(os.listdir(image_dir))
    lidar_files = sorted(os.listdir(lidar_dir))
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for img_file, lidar_file in zip(image_files, lidar_files):
            img_path = os.path.join(image_dir, img_file)
            lidar_path = os.path.join(lidar_dir, lidar_file)
            futures.append(executor.submit(process_image, calib, img_path, lidar_path, output_dir, interpolation_method, point_size))
        for future in concurrent.futures.as_completed(futures):
            future.result()

def main(root_dir, interpolation_method='linear', point_size=1):
    for subdir, dirs, files in os.walk(root_dir):
        if 'calib' in dirs and 'image_02' in dirs and 'velodyne_points' in dirs:
            print(f"Processing folder: {subdir}")
            process_folder(subdir, interpolation_method=interpolation_method, point_size=point_size)

root_dir = '/home/eddie/depth_estimation/monodepth2/sit_test_r3'
interpolation_method = 'linear'  # Adjust this value to 'linear', 'nearest', or 'cubic'
point_size = 2  # Adjust this value to set the size of the points
main(root_dir, interpolation_method=interpolation_method, point_size=point_size)


# 3. Visualization

#### 3.1 Project point cloud to img

In [ ]:
# points projection to img

import numpy as np

def load_velodyne_points(file_path):
    points = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
    return points

def load_calibration(file_path):
    calib = {}
    with open(file_path, 'r') as f:
        for line in f:
            key, value = line.split(':', 1)
            calib[key] = np.array([float(x) for x in value.split()])
    return calib


def project_velo_to_cam2(points, calib):
    P3 = calib['P3_intrinsic'].reshape(3, 3)
    R0_rect = calib['R0_rect'].reshape(3, 3)
    P3_extrinsic = calib['P3_extrinsic'].reshape(3, 4)
    
    points = points[:, :3]
    points = np.hstack((points, np.ones((points.shape[0], 1))))
    points = np.dot(points, P3_extrinsic.T)
    points = np.dot(points, R0_rect.T)
    
    points = points[points[:, 2] > 0]
    
    points = np.dot(points, P3.T)
    
    points[:, 0] /= points[:, 2]
    points[:, 1] /= points[:, 2]
    
    return points



import cv2

def visualize_points_on_image(image, points):
    for point in points:
        x, y = int(point[0]), int(point[1])
        if 0 <= x < image.shape[1] and 0 <= y < image.shape[0]:
            cv2.circle(image, (x, y), 1, (0, 255, 0), -1)
    return image

if __name__ == "__main__":

    lidar_file = '0.bin'
    calib_file = '0.txt'
    image_file = '0.png'
    output_image_file = '0_projected.png'
    
    points = load_velodyne_points(lidar_file)
    calib = load_calibration(calib_file)
    image = cv2.imread(image_file)
    
    projected_points = project_velo_to_cam2(points, calib)
    
    result_image = visualize_points_on_image(image, projected_points)
    
    cv2.imwrite(output_image_file, result_image)


#### 3.2 Generate inference result video

In [ ]:
# Make inference result video

import cv2
import os
import glob
import subprocess

# set image path
image_dir = '/home/eddie/depth_estimation/monodepth2/sit_data_inference/Three_way_Intersection/Three_way_Intersection_2/image_02/data'

jpg_path = os.path.join(image_dir, 'jpg')
jpeg_path = os.path.join(image_dir, 'jpeg')
jpg_files = sorted(glob.glob(os.path.join(jpg_path, '*.jpg')))
jpeg_files = sorted(glob.glob(os.path.join(jpeg_path, '*.jpeg')))

temp_dir = os.path.join(image_dir, 'temp_images')
os.makedirs(temp_dir, exist_ok=True)

for i, (jpg_file, jpeg_file) in enumerate(zip(jpg_files, jpeg_files)):

    img1 = cv2.imread(jpg_file)
    img1 = cv2.resize(img1, (1920, 1200)) 

    img2 = cv2.imread(jpeg_file)
    img2 = cv2.resize(img2, (1920, 1200)) 

    # concat images
    combined_img = cv2.vconcat([img1, img2])

    # save to tmp folder
    temp_image_path = os.path.join(temp_dir, f"{i:05d}.png")
    cv2.imwrite(temp_image_path, combined_img)

output_video_path = os.path.join(image_dir, 'output_video_10.mp4')
ffmpeg_cmd = [
    'ffmpeg', '-y', '-framerate', '10', '-i', os.path.join(temp_dir, '%05d.png'),
    '-c:v', 'libx264', '-pix_fmt', 'yuv420p', output_video_path
]
subprocess.run(ffmpeg_cmd)

# remove temp folder
# import shutil
# shutil.rmtree(temp_dir)

print(f"Video saved to {output_video_path}")


#### 3.3 Generate inference result video (Multi-threading)

In [ ]:
# Make inference result video (Multi-threading)

import cv2
import os
import glob
import subprocess
import concurrent.futures

image_dir = '/home/eddie/depth_estimation/monodepth2/sit_data_inference/Cafeteria/Cafeteria_1/image_02/data'

jpg_path = os.path.join(image_dir, 'jpg')
jpeg_path = os.path.join(image_dir, 'jpeg')
jpg_files = sorted(glob.glob(os.path.join(jpg_path, '*.jpg')))
jpeg_files = sorted(glob.glob(os.path.join(jpeg_path, '*.jpeg')))

# save to tmp folder
temp_dir = os.path.join(image_dir, 'temp_images')
os.makedirs(temp_dir, exist_ok=True)

def process_images(i, jpg_file, jpeg_file):
    img1 = cv2.imread(jpg_file)
    img1 = cv2.resize(img1, (1920, 1200)) 

    img2 = cv2.imread(jpeg_file)
    img2 = cv2.resize(img2, (1920, 1200)) 

    combined_img = cv2.vconcat([img1, img2])

    temp_image_path = os.path.join(temp_dir, f"{i:05d}.png")
    cv2.imwrite(temp_image_path, combined_img)

# multi-threading
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(process_images, i, jpg_file, jpeg_file)
        for i, (jpg_file, jpeg_file) in enumerate(zip(jpg_files, jpeg_files))
    ]
    for future in concurrent.futures.as_completed(futures):
        future.result()

output_video_path = os.path.join(image_dir, 'output_video_10.mp4')
ffmpeg_cmd = [
    'ffmpeg', '-y', '-framerate', '10', '-i', os.path.join(temp_dir, '%05d.png'),
    '-c:v', 'libx264', '-pix_fmt', 'yuv420p', output_video_path
]
subprocess.run(ffmpeg_cmd)

# remove tmp folder
# import shutil
# shutil.rmtree(temp_dir)

print(f"Video saved to {output_video_path}")
